In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-1 STARTS HERE##########################################

In [2]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [9]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e2], [], order=1)


ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.009532, evals: 57, done: 1, feasible: 0, sos: 3.13635, f: 0, ineq: 0, eq: 844.541 }
size of path: (1, 11)


0

RRT

In [10]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

for i in range(0, 2):
    ret = rrt.solve()
    print("rettttt: ", ret)
    path = ret.x
    print("path size:", path.shape[0])
    # display the path
    for t in range(0, path.shape[0] - 1):
        C.setJointState(path[t])
        C.view()
        time.sleep(0.002)

STACK9 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python() [0x50819f]
STACK8 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python(_PyObject_MakeTpCall
STACK7 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python() [0x4fc697]
STACK6 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x7b93fd) [0x7f398cbb93fd]
STACK5 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x809aa1) [0x7f398cc09aa1]
STACK4 rai::PathFinder::setProblem(rai::Configuration const&, rai::ArrayDouble const&, rai::ArrayDouble const&)
STACK3 RRT_PathFinder::RRT_PathFinder(ConfigurationProblem&, rai::ArrayDouble const&, rai::ArrayDouble const&, double, int, bool)
STACK2 ConfigurationProblem::query(rai::ArrayDouble const&)
STACK1 rai::Configuration::setJointState(rai::ArrayDouble const&)
STACK0 rai::LogToken::~LogToken()


== ERROR:kin.cpp:setJointState:634(-2) CHECK_EQ failed: '_q.N'=3 'N'=11 -- wrong joint state dimensionalities


RuntimeError: kin.cpp:setJointState:634(-2) CHECK_EQ failed: '_q.N'=3 'N'=11 -- wrong joint state dimensionalities

Trajectory Optimization via KOMO

In [11]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)


== ERROR:kin.cpp:setJointState:634(-2) CHECK_EQ failed: '_q.N'=3 'N'=11 -- wrong joint state dimensionalities


RuntimeError: kin.cpp:setJointState:634(-2) CHECK_EQ failed: '_q.N'=3 'N'=11 -- wrong joint state dimensionalities

STACK9 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python(PyEval_EvalCode
STACK8 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python() [0x591d92]
STACK7 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python(_PyEval_EvalFrameDefault
STACK6 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python() [0x50819f]
STACK5 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python(_PyObject_MakeTpCall
STACK4 /home/bornozlusapiens/miniconda3/envs/cs449/bin/python() [0x4fc697]
STACK3 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x7b93fd) [0x7f398cbb93fd]
STACK2 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x7fb59d) [0x7f398cbfb59d]
STACK1 rai::Configuration::setJointState(rai::ArrayDouble const&)
STACK0 rai::LogToken::~LogToken()


###########################################PART-1 ENDS HERE##########################################

###########################################PART-2 STARTS HERE##########################################

In [6]:
del komo
del C

In [12]:
C = ry.Config()
C.addFile('cargobot.g')
C.addFile('maze.g')
C.addFile('cargo.g')



C.view()


0

###########################################PART-2 ENDS HERE##########################################